In [1]:
import numpy as np
import pandas as pd
import re
import requests
from dotenv import load_dotenv
import os

# Read data

In [2]:
# Read data from csv
df = pd.read_csv("data/rental_prices_singapore.csv")

In [3]:
# Show dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5360 entries, 0 to 5359
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   name                           5360 non-null   object 
 1   address                        5011 non-null   object 
 2   price                          5360 non-null   object 
 3   size                           5360 non-null   object 
 4   bedrooms                       5360 non-null   object 
 5   bathrooms                      4941 non-null   float64
 6   property_type_furnishing_year  5360 non-null   object 
 7   mrt_distance                   4641 non-null   object 
 8   agent_description              5360 non-null   object 
dtypes: float64(1), object(8)
memory usage: 377.0+ KB


In [17]:
# Show top five rows
df.head()

,name,address,price,size,bedrooms,bathrooms,property_type_furnishing_year,mrt_distance,agent_description,latitude,longitude
0,"Brand new Attic Studio, in a Peranakan Conserv...",Lorong 34 Geylang,"3,000",400 sqft,1,1.0,\nApartment\nFully Furnished\n,NaN,One and only attic studio! Beautifully done up...,1.312952,103.887868
1,Astor,51C Lengkong Empat,"2,000",1130 sqft,Room,NaN,\nApartment\nFully Furnished\n,11 mins (810 m) to DT28 Kaki Bukit MRT,Comes with In House Maid,1.328820,103.912904
2,Springhill Terrace,Sunrise avenue,"7,400",3800 sqft,5,4.0,\nApartment\nFully Furnished\n,NaN,"Close to MRT and short drive to French, Austra...",1.389444,103.857002
3,704 Yishun Avenue 5,704 Yishun Avenue 5,"1,000",120 sqft,Room,NaN,\nApartment\nFully Furnished\n,9 mins (700 m) to NS13 Yishun MRT,Room for 1 or 2 single ladies,1.429261,103.828917
4,Espada,48 Saint Thomas Walk,"4,300",689 sqft,1,1.0,\nApartment\nFully Furnished\n,6 mins (420 m) to NS23 Somerset MRT,All units virtual online viewing available! An...,1.297356,103.836707


# Remove duplicates

In [5]:
# Diagnose duplicates
df.duplicated().value_counts()

False    5082
True      278
dtype: int64

In [6]:
# Remove duplicates
df = df.drop_duplicates().copy()

In [7]:
# Show dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5082 entries, 0 to 5359
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   name                           5082 non-null   object 
 1   address                        4754 non-null   object 
 2   price                          5082 non-null   object 
 3   size                           5082 non-null   object 
 4   bedrooms                       5082 non-null   object 
 5   bathrooms                      4680 non-null   float64
 6   property_type_furnishing_year  5082 non-null   object 
 7   mrt_distance                   4418 non-null   object 
 8   agent_description              5082 non-null   object 
dtypes: float64(1), object(8)
memory usage: 397.0+ KB


# Extract features

## Latitude and longitude

In [8]:
# Load environment variables from .env file
load_dotenv()

True

In [9]:
# Get Google Maps API key from .env 
google_maps_api_key = os.getenv("google_maps_api_key")

In [12]:
# Create function to get latitude and longitude from an address 
def get_latitude_longitude(address):
    # Base URL for the Google Maps Geocoding API
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    # Parameters for the API request
    params = {
        "address": f"{address}, Singapore",
        "key": google_maps_api_key
    }
    # Send API request and store the data
    response = requests.get(base_url, params=params)
    data = response.json()
    # Check if request was successful
    if data["status"] == "OK":
        # Extract latitude and longitude from the response
        location = data["results"][0]["geometry"]["location"]
        latitude = location["lat"]
        longitude = location["lng"]
    else:
        # Assign missing values and print error message if the request failed
        latitude = np.nan
        longitude = np.nan
        print(f"Geocoding request failed for {address}")
    # Return latitude and longitude
    return (latitude, longitude)

In [13]:
# Apply function to create latitude and longitude column
# df[["latitude", "longitude"]] = df["address"].apply(get_latitude_longitude).apply(pd.Series)

In [14]:
# Save enriched dataframe as csv
# df.to_csv("data/enriched_rental_prices_singapore.csv", index=False)

In [15]:
# Load enriched data
df = pd.read_csv("data/enriched_rental_prices_singapore.csv")

In [16]:
# Show dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5082 entries, 0 to 5081
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   name                           5082 non-null   object 
 1   address                        4754 non-null   object 
 2   price                          5082 non-null   object 
 3   size                           5082 non-null   object 
 4   bedrooms                       5082 non-null   object 
 5   bathrooms                      4680 non-null   float64
 6   property_type_furnishing_year  5082 non-null   object 
 7   mrt_distance                   4418 non-null   object 
 8   agent_description              5082 non-null   object 
 9   latitude                       5082 non-null   float64
 10  longitude                      5082 non-null   float64
dtypes: float64(3), object(8)
memory usage: 436.9+ KB


## Property type

In [ ]:
# Create function to extract property type 
def extract_type(string):
    if "Condominium" in string:
        return "Condominium"
    elif "Apartment" in string:
        return "Apartment"
    elif "HDB Flat" in string:
        return "HDB Flat"
    elif "Semi-Detached House" in string:
        return "Semi-Detached House"
    elif "Good Class Bungalow" in string:
        return "Good Class Bungalow"
    elif "Corner Terrace" in string:
        return "Corner Terrace"
    elif "Detached House" in string:
        return "Detached House"
    elif "Executive Condominium" in string:
        return "Executive Condominium"
    elif "Terraced House" in string:
        return "Terraced House"
    elif "Bungalow House" in string:
        return "Bungalow House"
    elif "Cluster House" in string:
        return "Cluster House"
    else:
        return np.nan

In [ ]:
# Apply function to create property type column
df["property_type"] = df["property_type_furnishing_year"].apply(extract_type)

In [ ]:
# Frequencies of property types
df["property_type"].value_counts()

## Furnishing

In [ ]:
# Create function to extract information about furnishing  
def extract_furnishing(string):
    if "Fully Furnished" in string:
        return "Fully Furnished"
    elif "Partially Furnished" in string:
        return "Partially Furnished"
    elif "Unfurnished" in string:
        return "Unfurnished"
    else:
        return np.nan

In [ ]:
# Apply function to create furnishing column
df["furnishing"] = df["property_type_furnishing_year"].apply(extract_furnishing)

In [ ]:
# Frequencies of furnishing
df["furnishing"].value_counts()

## Built year

In [ ]:
# Create function to extract built year 
def extract_year(string):
    year = re.search(r"\b\d{4}\b", string)
    if year:
        return year.group()
    else:
        return np.nan

In [ ]:
# Apply function to create built year column
df["year"] = df["property_type_furnishing_year"].apply(extract_year).astype("Int32")

In [ ]:
# Frequencies of built year
df["year"].value_counts()

In [ ]:
# Delete "property_type_furnishing_year" column
df.drop("property_type_furnishing_year", axis=1, inplace=True)

## Meters to MRT

In [ ]:
# Extract MRT distance in meters
df["meters_to_mrt"] = df["mrt_distance"].str.split(r"m\)").str[0].str.split(r"\(").str[1].astype("Int32")

In [ ]:
# Delete "mrt_distance" column
df.drop("mrt_distance", axis=1, inplace=True)

## Renovated

In [ ]:
# Create function to extract information about renovation  
def extract_renovated(string):
    if "renovated" in string.lower() or "renovation" in string.lower():
        return True
    else:
        return False

In [ ]:
# Apply function to create renovated column
df["renovated"] = df["agent_description"].apply(extract_renovated)

## Pool

In [ ]:
# Create function to extract information about pool  
def extract_pool(string):
    if "pool" in string.lower():
        return True
    else:
        return False

In [ ]:
# Apply function to create pool column
df["pool"] = df["agent_description"].apply(extract_pool)

## Sea view

In [ ]:
# Create function to extract information about sea view  
def extract_sea_view(string):
    if "seaview" in string.lower() or "sea view" in string.lower():
        return True
    else:
        return False

In [ ]:
# Apply function to create sea view column
df["sea_view"] = df["agent_description"].apply(extract_sea_view)

## Penthouse

In [ ]:
# Create function to extract penthouse information 
def extract_penthouse(string):
    if "penthouse" in string.lower():
        return True
    else:
        return False

In [ ]:
# Apply function to create penthouse column
df["penthouse"] = df["agent_description"].apply(extract_penthouse)

In [ ]:
# Delete "agent_description" column
# df.drop("agent_description", axis=1, inplace=True)

# Convert data types

In [ ]:
# Convert price from str to int
df["price"] = df["price"].str.replace(",", "").astype("Int32")

In [ ]:
# Convert size (sqft) from str to int
df["size"] = df["size"].str.split("sqft").str[0].astype("Int32")

In [ ]:
# Convert bathrooms from str to int
df["bathrooms"] = df["bathrooms"].astype("Int32")

# Check data

In [ ]:
# Show dataframe info
df.info()

In [ ]:
# Explore properties with missing address
filtered_names = df.loc[pd.isna(df["address"]), "agent_description"]

for name in filtered_names:
    print(name)

In [ ]:
# Explore properties with missing address
df.loc[pd.isna(df["address"])]

In [ ]:
# Show top and bottom five rows
df

# To Do
- Get distance to attractive locations in Singapore (e.g., marina bay, orchard, botanic garden)
- Handle missing values
  - address
  - bathrooms
  - mrt distance
  - furnishing
  - built year